In [169]:
# This is the xgboost implementation to select the most important genes from the 32,830 given. 
#Hyperparameter tuning is done using Randomized SearchCV. Xgboost model is fitted on these parameters.
#The most important genes returned by this model is fitted into ElasticNet regression in R.(GLMNET and LARS package)
#The current best accuracy is RMSE of 5.4, using GLMNET. 
#This score can be further reduced through more efficent and computationally rigorous use of Randomized SearchCV, using ".best_score_" as the evaluation metric.  

In [170]:

from sklearn.model_selection import train_test_split    
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt
import xgboost as xgb     
import numpy as np         
import pandas as pd 


In [171]:
#setting up the data.
# Train==1 dataset is split into train and test in 80:20 ratio.

anoSC1 = pd.read_csv('/Users/User/Downloads/anoSC1_v11_nokey.csv.xls')  #annotation file of patients,GA, batch,train/test 
anoSC1 = anoSC1.set_index('SampleID')
geneExp = pd.read_csv('/Users/User/Downloads/csv_genes.csv')            # gene expression data of 32,830 genes, of 735 patients
geneExp = geneExp.set_index('Unnamed: 0')
geneExp = geneExp.T
geneExp = pd.concat([geneExp, anoSC1], axis=1, sort=False)
geneExp = geneExp.drop(['Batch','Set','Platform'],axis=1)    #irrelevant columns dropped
#geneExp
geneExpdata= geneExp.loc[geneExp['Train'] == 1]
geneExpdata
X = geneExpdata.iloc[:,0:32830]
y = geneExpdata.iloc[:,-2]





In [3]:
#preparing the training and test datasets in the aforementioned ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
reg_xgb = xgb.XGBRegressor(objective='reg:squarederror',seed=42)
reg_xgb.fit(X_train,
           y_train,
           verbose=True,
            early_stopping_rounds=10,
            eval_metric='rmse',
            eval_set=[(X_test,y_test)],
           )

[0]	validation_0-rmse:19.12665
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:14.41761
[2]	validation_0-rmse:11.53431
[3]	validation_0-rmse:9.89808
[4]	validation_0-rmse:8.86578
[5]	validation_0-rmse:8.23018
[6]	validation_0-rmse:8.10360
[7]	validation_0-rmse:8.05049
[8]	validation_0-rmse:8.05922
[9]	validation_0-rmse:8.08173
[10]	validation_0-rmse:8.10060
[11]	validation_0-rmse:8.13783
[12]	validation_0-rmse:8.16622
[13]	validation_0-rmse:8.18311
[14]	validation_0-rmse:8.20486
[15]	validation_0-rmse:8.19797
[16]	validation_0-rmse:8.21230
[17]	validation_0-rmse:8.21771
Stopping. Best iteration:
[7]	validation_0-rmse:8.05049



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [85]:
params={
 "learning_rate"    : [ 0.05, 0.10, 0.20,0.5] ,
 "max_depth"        : [ 2,3,5,7],
 "min_child_weight" : [ 3, 5, 7,9],
 "gamma"            : [0,0.1,0.5,0.75,1,3,5],
 #"colsample_bytree" : [ 0.3, 0.4, 0.5],
  "reg_lambda"      : [0,0.5,1,2,5],
    "subsample"     : [0.7,0.8,0.9],

}


In [86]:
optimal_params = RandomizedSearchCV(
estimator = xgb.XGBRegressor(objective='reg:squarederror',seed=42,
                             verbose=0,cv=5,
                             colsample_bytree= 0.4
                            ),
    param_distributions=params,
    n_iter=20,
    n_jobs=-1,
    scoring ='neg_root_mean_squared_error',
    verbose=0,
    cv=5
                              
)

In [87]:
optimal_params.fit(X_train,
                  y_train,
                  early_stopping_rounds=20,
                  eval_metric='rmse',
                  eval_set=[(X_test,y_test)],
                    verbose=True)

#print(optimal_params.best_params_)



[23:56:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { cv, verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:23.90999
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:21.73021
[2]	validation_0-rmse:19.80140
[3]	validation_0-rmse:18.08213
[4]	validation_0-rmse:16.60447
[5]	validation_0-rmse:15.19585
[6]	validation_0-rmse:14.02967
[7]	validation_0-rmse:12.94711
[8]	validation_0-rmse:12.02000
[9]	validation_0-rmse:11.19351
[10]	validation_0-rmse:10.56595
[11]	validation_0-rmse:9.95693
[12]	validation_0-rmse:9.51762
[13]	validation_0-rmse:9.11019
[14]	validation_0-rmse:8.72674
[15]	validation_0-rmse:8.35591
[16]	validation_0-rmse:8.13129
[17]	validation_0-

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.4, cv=5,
                                          gamma=None, gpu_id=None,
                                          importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=Non...
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5],
                                        'gamma': [0, 0.1, 0.5, 0.75, 1, 3, 5],
                                  

In [90]:
opt_params = optimal_params.best_params_

{'subsample': 0.8, 'reg_lambda': 0.5, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 3, 'colsample_bytree': 0.4}


In [91]:
optimal_params.best_score_

-6.710798434309567

In [96]:
reg_xgb=xgb.XGBRegressor(seed=42,
                    objective='reg:squarederror',
                    subsample= 0.8, reg_lambda= 0.5,
                    min_child_weight= 5, max_depth= 3, learning_rate= 0.1, gamma= 3, colsample_bytree= 0.4)
                      
reg_xgb.fit(X_train,
           y_train,
           verbose=True,
            early_stopping_rounds=10,
            eval_metric='rmse',
            eval_set=[(X_test,y_test)],
           )

[0]	validation_0-rmse:23.90999
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:21.73021
[2]	validation_0-rmse:19.80140
[3]	validation_0-rmse:18.08213
[4]	validation_0-rmse:16.60447
[5]	validation_0-rmse:15.19585
[6]	validation_0-rmse:14.02967
[7]	validation_0-rmse:12.94711
[8]	validation_0-rmse:12.02000
[9]	validation_0-rmse:11.19351
[10]	validation_0-rmse:10.56595
[11]	validation_0-rmse:9.95693
[12]	validation_0-rmse:9.51762
[13]	validation_0-rmse:9.11019
[14]	validation_0-rmse:8.72674
[15]	validation_0-rmse:8.35591
[16]	validation_0-rmse:8.13129
[17]	validation_0-rmse:7.96544
[18]	validation_0-rmse:7.80719
[19]	validation_0-rmse:7.70599
[20]	validation_0-rmse:7.56062
[21]	validation_0-rmse:7.44782
[22]	validation_0-rmse:7.38979
[23]	validation_0-rmse:7.30558
[24]	validation_0-rmse:7.23991
[25]	validation_0-rmse:7.16402
[26]	validation_0-rmse:7.11505
[27]	validation_0-rmse:7.10574
[28]	validation_0-rmse:7.06229
[29]	validation_0-rmse:7.04057
[30]

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=3, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=3,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0,
             reg_lambda=0.5, scale_pos_weight=1, seed=42, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [133]:
Imp_feature = reg_xgb.get_booster().get_score(importance_type = 'gain')
Imp_feature = list(Imp_feature.keys())


In [139]:
x_train=X_train[Imp_feature]
x_test=X_test[Imp_feature]
x_train.head()

,51108_at,101927924_at,2359_at,114793_at,105369454_at,10321_at,79980_at,100506084_at,3495_at,5027_at,...,100379661_at,100271076_at,100128126_at,100337591_at,5746_at,28595_at,100302203_at,8220_at,101930665_at,101930149_at
Tarca_892_P10D04,9.112816,4.550605,4.124514,3.905545,2.739592,3.627982,4.684861,2.450111,7.715769,6.178199,...,5.256638,3.074483,3.180507,2.830858,3.378539,7.061327,3.678235,7.081346,3.053669,3.405693
Tarca_587_P7C02,8.958542,4.706246,4.524360,4.064426,2.605540,4.457869,4.691887,3.045944,7.188265,5.790084,...,5.273639,3.031451,3.391637,2.758438,3.626577,7.367961,4.516202,7.125210,3.478968,3.187149
Tarca_202_P3B02,8.883469,4.698160,4.377986,3.931385,2.628411,4.066394,4.510200,2.529345,7.393275,5.962908,...,5.508177,3.399706,3.259721,2.806554,3.480615,7.355895,4.094447,7.146316,3.409484,3.067205
Tarca_217_P3A04,9.073711,4.984694,4.289244,3.865520,2.611236,5.661098,4.635563,2.897117,7.255582,5.620907,...,5.445485,2.800814,3.383102,2.563158,3.450020,7.000798,4.745016,7.197676,2.739291,3.140762
Tarca_909_P10E06,9.176856,4.588777,4.450763,4.045642,2.817271,5.436464,4.587122,1.899637,8.738293,6.118985,...,5.677633,3.220946,3.372158,2.950113,3.520737,7.246644,4.065140,6.915086,3.285037,3.088567


In [ ]:
# Imp_feature list is sent to R, for fitting elasticnet regression on the selected 336 genes.
#lowest RMSE score till now is 5.4